In [1]:
import pandas as pd
import numpy as np
from io import StringIO

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
widths=[3,1,3]

df=pd.read_fwf('LOG01.txt',widths=widths, names=['1','2','3'])
print(df)

         1  2    3
0      516     510
1      516     510
2      516     510
3      516     510
4      516     510
...    ... ..  ...
20117  504     515
20118  504     515
20119  504     515
20120  504     515
20121  504      51

[20122 rows x 3 columns]


In [4]:
df.drop(columns='2',inplace=True)


In [5]:
df

,1,3
0,516,510
1,516,510
2,516,510
3,516,510
4,516,510
...,...,...
20117,504,515
20118,504,515
20119,504,515
20120,504,515


In [6]:

index=df.index/1000
print(index)


Float64Index([   0.0,  0.001,  0.002,  0.003,  0.004,  0.005,  0.006,  0.007,
               0.008,  0.009,
              ...
              20.112, 20.113, 20.114, 20.115, 20.116, 20.117, 20.118, 20.119,
               20.12, 20.121],
             dtype='float64', length=20122)


In [9]:
np.mean(df["1"][:2000])

516.977

In [10]:
3.3/1024

0.00322265625

In [54]:
from scipy import fft
from scipy.fft import fft, fftfreq

In [15]:
fig=px.scatter(df, y=[(df['3']-512)/21,(df['1']-512)/21])
fig.show()

In [58]:
def get_fft(df):

    label=df.index/1000
    
    N=len(df)
    fs = len(df)/(label[-1]-label[0])
    
    x_plot= fftfreq(N, 1/fs)[:N//2]
    
    df_fft = pd.DataFrame()
    df_phase = pd.DataFrame()
    for name in df.columns:
        yf = fft(df[name].values) 
        
        y_plot= 2.0/N * np.abs(yf[0:N//2])
        
        df_fft = pd.concat([df_fft,
                            pd.DataFrame({'Frequency (Hz)':x_plot[1:],
                                          name:y_plot[1:]}).set_index('Frequency (Hz)')],axis=1)
    
    return df_fft

In [59]:
x2=get_fft(df)


In [60]:
print(x2)

                       1         3
Frequency (Hz)                    
0.049699        3.646569  5.279982
0.099399        1.583209  1.469493
0.149098        1.886830  1.736805
0.198797        0.833915  3.248726
0.248497        0.976793  1.461069
...                  ...       ...
499.776353      0.005176  0.105891
499.826052      0.007570  0.029831
499.875752      0.003093  0.032483
499.925451      0.015647  0.010860
499.975150      0.017659  0.101753

[10060 rows x 2 columns]


In [61]:


fig = make_subplots()
fig.add_trace(go.Scatter(x=x2.index[1:], y=x2['1'], name='stock' 
                         ))
fig.add_trace(go.Scatter(x=x2.index[1:], y=x2['3'], name='HF' 
             ))

            

fig.update_layout(
    #title=f"FFT's {run[:-4]} ",
    xaxis_title="Frequency (Hz)",) 
fig.update_traces(marker_line_width=0.1, opacity=0.7,
                  selector=dict(type='scatter'))                
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig.show()